### Notes
We might need to remove the common words in the corpus beside the stop words as those are common cross all corpus

In [1]:
import pandas as pd
from matplotlib import pyplot as plt

utterances = pd.read_csv('./temp_data/utterances.csv')
utterances['actionbody'] = utterances['actionbody'].astype(str)
sample_df = utterances
sample_df

,id,issueid,created,author,actionbody,utr_seq,is_private,comment_seq
0,211718001.0,1004285.0,2022-01-01 18:54:38+00:00,ae.edw.03,greetings,0,0,0
1,211718001.0,1004285.0,2022-01-01 18:54:38+00:00,ae.edw.03,we do not know why if the kube certificates we...,1,0,0
2,211718001.0,1004285.0,2022-01-01 18:54:38+00:00,ae.edw.03,it is now again showing expired on 26-11-2021....,2,0,0
3,211718001.0,1004285.0,2022-01-01 18:54:38+00:00,ae.edw.03,kubectl services are not coming up on staging.,3,0,0
4,211718001.0,1004285.0,2022-01-01 18:54:38+00:00,ae.edw.03,we need to renew the certificates.,4,0,0
...,...,...,...,...,...,...,...,...
293671,1115987.0,1010829.0,2023-01-10 22:22:45+00:00,u447,ph_user,3,0,16
293672,1115997.0,1010829.0,2023-01-10 23:06:13+00:00,u447,greetings,0,0,17
293673,1115997.0,1010829.0,2023-01-10 23:06:13+00:00,u447,please consider this ticket as closed,1,0,17
293674,1115997.0,1010829.0,2023-01-10 23:06:13+00:00,u447,regards,2,0,17


In [2]:
len(utterances)

293676

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(
    max_df=1.0,
    # min_df=0.005,
    max_features=None,
    stop_words=None,
    # token_pattern=r'(?u)(\b\w\w+\b|#\b\w\w+\b#)'
)
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(sample_df['actionbody'])
tfidf
vocabulary = tfidf_vectorizer.vocabulary_

print(f'number of features: {len(vocabulary)}')


number of features: 32892


In [21]:
voc = [c for c in vocabulary.keys()]
voc.sort()
# words

In [22]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words
import nltk as nltk

nltk.download('wordnet')
 
lemmatizer = WordNetLemmatizer()

unknown_words = []
for w in voc:
    if w not in words.words() and lemmatizer.lemmatize(w) not in words.words():
        unknown_words.append(w)

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['00',
 '000',
 '0000',
 '00000',
 '000000',
 '00000000',
 '0000000000',
 '000000000000',
 '0000000000000',
 '0000000000000000',
 '000000000000000000000000000000000000000000000000',
 '0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '0000000000000000000000000000000a',
 '00000000000000000101',
 '00000000000000000326',
 '00000000000360092010400000000000728375500140920221409202201101',
 '00000000000360092010400000000000728375500140920221409202201101n',
 '00000000000560092010400000000000741034500071120220711202200101',
 '0000000004178635',
 '000000012339942000000000000000000000100',
 '000000012339942000000000000000000000100success',
 '000000012623917000000000000000000000100success',
 '000000022656235001',
 '00000050',
 '00000085',
 '0000008b',
 '0000008c',
 '000001',
 '00000100000',
 '00000100001113168',
 '0000010000xxxxx140',
 '00000133',
 '000002338163',
 '000002608745',
 '000002608759',
 '000002608992',
 '000002609010',
 '000003',
 '000006',
 '

In [23]:
ds_voc = pd.DataFrame(data=unknown_words,columns=["word"])
ds_voc['type'] = pd.NA
ds_voc.set_index('word',inplace=True)
ds_voc.to_csv('./temp_data/utterances_voc.csv')
print('done')

done
